## 1. 训练集和检验集

在应用机器学习算法前，一般将数据集划分为训练集(training set)和检验集(test set)，训练集用于拟合模型，检验集用于评估预测能力。

机器学习模型往往包含很多参数，如果不使用检验集而是直接评估样本内的预测精度，会受到过度拟合的影响。所谓过度拟合，就是模型找到了本来不存在的规律，利用检验集来校验模型能规避这一点。

![train test split](../pic/train_test_split.png)

sklearn实现：**train_test_split(X, y, test_size, random_state)**

* X: 特征矩阵
* y: 目标向量
* test_size: 检验集大小(比例)，通常选择0.3，0.25，0.2等
* random_state: Numpy RandomState对象或代表随机数种子的整数，由于划分是随机的，为了重复实验过程，应该使用统一的随机数种子。

使用IRIS数据集。

In [5]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

iris = load_iris()
X, y = iris.data, iris.target

iris_df = pd.DataFrame(X, columns=iris["feature_names"])
iris_df["target"] = y

iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [6]:
# 先设置随机数种子，目标是复现试验结果
rs = 123

# 调用train_test_split，随机划分训练集和检验集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=rs)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# 创建模型，这里使用KNN作为说明
model = KNeighborsClassifier(n_neighbors=5)

# 训练集
model.fit(X_train, y_train)

# 评估训练集和检验集的预测精度
score_in_sample = model.score(X_train, y_train)
score_out_sample = model.score(X_test, y_test)

print("in-sample mean accuracy: %.4f" % score_in_sample)
print("out-sample mean accuracy: %.4f" % score_out_sample)

(105, 4)
(45, 4)
(105,)
(45,)
in-sample mean accuracy: 0.9810
out-sample mean accuracy: 0.9778


## 2. 训练/检验的缺陷

将数据集划分为训练集和检验集存在一个缺陷，即评估结果的不一致性。划分过程通常是随机的，给定不同的训练样本和测试样本（例如调整随机数种子）会得到不同的预测评分，每一个的检验集的精度评估都是模型真实预测能力的有偏估计，不能作为绝对的评估指标。

In [7]:
# 通过控制随机数种子，获得不同的(train,test)组合，分别拟合模型并计算检验集的预测评分
random_state_list = range(1,21)
score_list = []

for random_state in random_state_list:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)
    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    score_list.append(score)
    print("score: %.4f" % score)

print("mean score: %.4f" % np.mean(score_list))

score: 0.9778
score: 1.0000
score: 0.9556
score: 0.9778
score: 0.9556
score: 0.9778
score: 0.9111
score: 0.9556
score: 1.0000
score: 0.9778
score: 0.9778
score: 0.9778
score: 0.9111
score: 0.9556
score: 0.9778
score: 0.9556
score: 0.9556
score: 1.0000
score: 0.9556
score: 0.9556
mean score: 0.9656


## 3. 交叉验证

为了克服训练/检验划分的缺陷，并得到模型预测能力更准确的评估，需要引入一个新的概念：K折交叉验证。

什么是交叉验证(cross-validation)？

给定样本数据，划分训练集和检验集，在训练集上训练模型，在检验集上测试，然后不断重复这个过程，得到多个预测评分，取预测评分的均值作为真实预测能力的无偏估计。

1. 将数据分为k个组，简称k折(k折交叉验证)，既可以按顺序划分，也可以随机划分。
2. 随机抽取一折作为检验集，剩下的(k-1)折作为训练集，在训练集上拟合模型，在检验集上测试，记录预测评分。
3. 将这种过程重复k次，每次迭代的训练集和测试集要求不相同。
4. 计算预测评分的均值，作为模型真实预测能力的评估。

![cross validation](../pic/cross_validation.png)

### 3.1 用迭代器实现

使用迭代器控制整个交叉验证的过程，迭代器的主要功能是创建索引，即生成折数。

sklearn.model_selection模块提供了很多迭代器，例如KFold, StratifiedKFold, LeaveOneOut等。

迭代器对象包含k个元素，每个元素都是长度为2的元组，元组的第一个元素包含训练集的索引号，第二个元素包含检验集的索引号，利用这些索引号划分训练集和检验集。

In [10]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5,  # 折数
              shuffle=True,  # 随机分配
              random_state=1)

for i, (train, test) in enumerate(kfold.split(X)):
    print("fold %d" % (i + 1))
    print("="*50)
    print("training index: ", train)
    print("test index: ", test)
    print("="*50, "\n")

fold 1
training index:  [  0   1   2   3   4   6   7   8   9  10  11  12  13  15  17  18  20  21
  22  23  24  25  26  27  28  30  32  34  36  37  38  39  41  43  45  46
  47  48  49  50  52  53  54  55  57  58  59  60  61  62  63  64  65  67
  68  69  70  71  72  74  76  79  80  81  82  83  85  86  87  88  89  91
  93  95  96  97 100 101 103 104 105 106 107 108 109 110 111 112 113 114
 115 116 117 118 119 121 122 123 124 126 127 128 129 130 132 133 134 135
 136 137 138 139 140 142 143 144 145 147 148 149]
test index:  [  5  14  16  19  29  31  33  35  40  42  44  51  56  66  73  75  77  78
  84  90  92  94  98  99 102 120 125 131 141 146]

fold 2
training index:  [  0   1   2   3   5   6   7   8   9  10  11  12  13  14  15  16  19  20
  21  22  23  24  25  26  27  29  30  31  32  33  34  35  37  38  40  41
  42  43  44  46  47  49  50  51  52  55  56  57  60  61  62  63  64  65
  66  67  68  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84
  86  87  88  89  90  92  93  94  9

用10折交叉验证检验一个简单的模型，使用iris数据集。

In [12]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier

X = iris.data
y = iris.target

# 创建迭代器
kfold = KFold(n_splits=10,
              shuffle=True,
              random_state=123)

# 根据迭代器先划分训练集和检验集，然后拟合模型和评估性能
model = KNeighborsClassifier(n_neighbors=5)
score_list = []
for i, (idx_train, idx_test) in enumerate(kfold.split(X)):
    X_train, y_train = X[idx_train], y[idx_train]
    X_test, y_test = X[idx_test], y[idx_test]
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print("iteration %d: score = %.4f" % (i + 1, score))
    score_list.append(score)
    
score_mean = np.mean(score_list)
score_std = np.std(score_list)
print("average score = %.4f" % score_mean)
print("std of scores = %.4f" % score_std)

iteration 1: score = 0.9333
iteration 2: score = 1.0000
iteration 3: score = 0.9333
iteration 4: score = 1.0000
iteration 5: score = 1.0000
iteration 6: score = 0.9333
iteration 7: score = 1.0000
iteration 8: score = 1.0000
iteration 9: score = 1.0000
iteration 10: score = 0.8667
average score = 0.9667
std of scores = 0.0447


### 3.2 cross_val_score

用sklearn.model_selection.cross_val_score实现自动化交叉验证。

cross_val_score()省略了交叉验证的很多细节，如果提供了目标变量，在拆分样本时每一折的目标变量的标签所占的比例与y向量保持一致，即实现了分层抽样。如果要使用方法1实现相同的结果，需要创建StratifiedKFold。

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5)

scores = cross_val_score(
    estimator=model,  # 要评估的模型
    X=X,  # 特征矩阵
    y=y,  # 目标向量
    scoring="accuracy",  # 使用准确率作为预测评分
    cv=10,  # 交叉验证折数
    n_jobs=-1  # 使用全部CPU，平行运算
)

print(scores)
print("average score = %.4f" % np.mean(scores))
print("std of scores = %.4f" % np.std(scores))

[1.         0.93333333 1.         1.         0.86666667 0.93333333
 0.93333333 1.         1.         1.        ]
average score = 0.9667
std of scores = 0.0447
